# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [4]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [26]:
sinasc_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ORIGEM        27028 non-null  int64  
 1   CODESTAB      26913 non-null  float64
 2   CODMUNNASC    27028 non-null  int64  
 3   LOCNASC       27028 non-null  int64  
 4   IDADEMAE      27028 non-null  int64  
 5   ESTCIVMAE     26711 non-null  float64
 6   ESCMAE        26716 non-null  object 
 7   CODOCUPMAE    24121 non-null  float64
 8   QTDFILVIVO    25455 non-null  float64
 9   QTDFILMORT    24930 non-null  float64
 10  CODMUNRES     27028 non-null  int64  
 11  GESTACAO      25796 non-null  object 
 12  GRAVIDEZ      26949 non-null  object 
 13  PARTO         26979 non-null  object 
 14  CONSULTAS     27028 non-null  int64  
 15  DTNASC        27028 non-null  object 
 16  HORANASC      27007 non-null  float64
 17  SEXO          27024 non-null  object 
 18  APGAR1        26932 non-nu

# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [6]:
# Subconjunto dos dados relevantes
dados_idades = sinasc_raw[['munResNome', 'IDADEMAE', 'IDADEPAI']]

In [7]:
# Remova linhas com valores ausentes nas colunas de interesse
dados_idades = dados_idades.dropna(subset=['IDADEMAE', 'IDADEPAI'])

In [8]:
# Converta as colunas de idade para o tipo numérico
dados_idades['IDADEMAE'] = pd.to_numeric(dados_idades['IDADEMAE'], errors='coerce')
dados_idades['IDADEPAI'] = pd.to_numeric(dados_idades['IDADEPAI'], errors='coerce')

In [10]:
# Calcule a média por município
media_idade_mae = dados_idades.groupby('munResNome')['IDADEMAE'].mean()
media_idade_pai = dados_idades.groupby('munResNome')['IDADEPAI'].mean()

In [11]:
# Resultados
print("Média de Idade das Mães por Município:")
print(media_idade_mae)

print("\nMédia de Idade dos Pais por Município:")
print(media_idade_pai)

Média de Idade das Mães por Município:
munResNome
Alta Floresta D'Oeste        25.892857
Alto Alegre dos Parecis      24.679104
Alto Paraíso                 22.833333
Alvorada D'Oeste             26.825243
Ariquemes                    28.133333
Buritis                      24.750000
Cabixi                       32.166667
Cacaulândia                  31.000000
Cacoal                       26.978605
Campo Novo de Rondônia       24.900000
Candeias do Jamari           23.684211
Castanheiras                 28.071429
Cerejeiras                   28.361111
Chupinguaia                  26.888889
Colorado do Oeste            31.161290
Corumbiara                   29.000000
Costa Marques                25.250000
Cujubim                      27.400000
Espigão D'Oeste              26.808176
Governador Jorge Teixeira    25.555556
Guajará-Mirim                26.058824
Itapuã do Oeste              31.375000
Jaru                         28.450704
Ji-Paraná                    27.282819
Machadinho D'O

### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [28]:
# Subconjunto dos dados relevantes
dados_peso_escolaridade = sinasc_raw[['DTNASC', 'PESO', 'ESCMAE', 'SEXO']]

In [29]:
# Remova linhas com valores ausentes nas colunas de interesse
dados_peso_escolaridade = dados_peso_escolaridade.dropna(subset=['DTNASC', 'PESO', 'ESCMAE', 'SEXO'])

In [30]:
# Converta a coluna de data de nascimento para o tipo datetime
dados_peso_escolaridade['DTNASC'] = pd.to_datetime(dados_peso_escolaridade['DTNASC'], errors='coerce')

In [31]:
# Filtra os dados para o dia do meu aniversario, 18 de maio
dados_peso_escolaridade = dados_peso_escolaridade[dados_peso_escolaridade['DTNASC'].dt.day == 18]
dados_peso_escolaridade = dados_peso_escolaridade[dados_peso_escolaridade['DTNASC'].dt.month == 5]

In [32]:
# Converte a coluna de escolaridade para uma faixa de escolaridade
faixas_escolaridade = {
    'Nenhuma': '1. Nenhuma',
    '1 a 3 anos': '2. 1 a 3 anos',
    '4 a 7 anos': '3. 4 a 7 anos',
    '8 a 11 anos': '4. 8 a 11 anos',
    '12 e mais': '5. 12 e mais',
    'Ignorado': '0. Ignorado'
}

dados_peso_escolaridade['FAIXA_ESCOLARIDADE'] = dados_peso_escolaridade['ESCMAE'].map(faixas_escolaridade)

In [33]:
# Calcula o peso médio por sexo e faixa de escolaridade
peso_medio_por_escolaridade = dados_peso_escolaridade.groupby(['SEXO', 'FAIXA_ESCOLARIDADE'])['PESO'].mean().reset_index()

In [34]:
print(peso_medio_por_escolaridade)

        SEXO FAIXA_ESCOLARIDADE         PESO
0   Feminino      2. 1 a 3 anos  2882.500000
1   Feminino      3. 4 a 7 anos  3373.142857
2   Feminino     4. 8 a 11 anos  3157.894737
3  Masculino      3. 4 a 7 anos  3270.000000
4  Masculino     4. 8 a 11 anos  3190.388889


### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [37]:
# Use a função value_counts() para contar a frequência de nascimentos por município
contagem_municipios = sinasc_raw['munResNome'].value_counts()

In [38]:
# município com a menor contagem
menos_nascimentos = contagem_municipios.idxmin()

In [39]:
print(f"O município com menos nascimentos em 2019 foi: {menos_nascimentos}")

O município com menos nascimentos em 2019 foi: Município ignorado - RO


In [41]:
# Agrupe por município e calcule as estatísticas descritivas da idade da mãe
estatisticas_idade_mae = sinasc_raw.groupby('munResNome')['IDADEMAE'].aggregate(['mean', 'max', 'min'])
estatisticas_idade_mae

,mean,max,min
munResNome,,,
Alta Floresta D'Oeste,25.991826,53,14
Alto Alegre dos Parecis,24.844156,41,14
Alto Paraíso,24.960177,42,11
Alvorada D'Oeste,25.770732,44,15
Ariquemes,25.607866,44,13
Buritis,25.559177,44,14
Cabixi,26.037500,39,13
Cacaulândia,25.546667,42,16
Cacoal,26.890102,47,14


In [42]:
# Agrupe por município e calcule as estatísticas descritivas da idade do pai
estatisticas_idade_mae = sinasc_raw.groupby('munResNome')['IDADEPAI'].aggregate(['mean', 'max', 'min'])
estatisticas_idade_mae

,mean,max,min
munResNome,,,
Alta Floresta D'Oeste,29.548701,58.0,17.0
Alto Alegre dos Parecis,29.179104,50.0,16.0
Alto Paraíso,28.833333,49.0,21.0
Alvorada D'Oeste,30.757282,52.0,17.0
Ariquemes,32.466667,51.0,20.0
Buritis,30.812500,49.0,18.0
Cabixi,34.333333,47.0,19.0
Cacaulândia,36.333333,50.0,28.0
Cacoal,30.931854,66.0,17.0


### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



In [45]:
# Converta a coluna DTNASC para o tipo datetime
sinasc_raw['DTNASC'] = pd.to_datetime(sinasc_raw['DTNASC'], errors='coerce')

In [46]:
# Subconjunto dos dados para considerar apenas nascimentos no mês de março
nascimentos_marco = sinasc_raw[sinasc_raw['DTNASC'].dt.month == 3]

In [47]:
# Encontre o município com mais nascimentos em março
municipio_com_mais_nascimentos = nascimentos_marco['munResNome'].value_counts().idxmax()

In [48]:
print(f'O município com mais nascimentos em março foi: {municipio_com_mais_nascimentos}')

O município com mais nascimentos em março foi: Porto Velho


In [49]:
# Subconjunto dos dados para considerar apenas nascimentos em Porto Velho
nascimentos_porto_velho = sinasc_raw[sinasc_raw['munResNome'] == 'Porto Velho']

In [51]:
# Calcule a média, máximo e mínimo da quantidade de filhos vivos
media_filhos_vivos = nascimentos_porto_velho['QTDFILVIVO'].mean()
max_filhos_vivos = nascimentos_porto_velho['QTDFILVIVO'].max()
min_filhos_vivos = nascimentos_porto_velho['QTDFILVIVO'].min()

In [52]:
print(f'Média de filhos vivos em Porto Velho: {media_filhos_vivos:.2f}')
print(f'Máximo de filhos vivos em Porto Velho: {max_filhos_vivos}')
print(f'Mínimo de filhos vivos em Porto Velho: {min_filhos_vivos}')

Média de filhos vivos em Porto Velho: 1.17
Máximo de filhos vivos em Porto Velho: 12.0
Mínimo de filhos vivos em Porto Velho: 0.0


In [63]:
# Subconjunto dos dados para considerar apenas nascimentos em Porto Velho
nascimentos_porto_velho = sinasc_raw[sinasc_raw['munResNome'] == 'Porto Velho']

In [65]:
# Agrupe por 'munResNome' e calcule a média, máximo e mínimo da idade do pai e da mãe
resultados_idade_pais_maes_porto_velho = nascimentos_porto_velho.groupby('munResNome').agg({
    'IDADEPAI': ['mean', 'max', 'min'],
    'IDADEMAE': ['mean', 'max', 'min']
})

In [66]:
print(resultados_idade_pais_maes_porto_velho)

              IDADEPAI              IDADEMAE        
                  mean   max   min      mean max min
munResNome                                          
Porto Velho  32.352679  65.0  16.0  26.31267  47  12


### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

- A maior media da idade dos pais eh Itapuã do Oeste (42)
- A menor media da idade dos pais eh Município ignorado - RO (24)
- A media da idade das maes por estado ficam entre 24 e 27 anos. Muito menos variacao que a media de idade dos pais.

In [6]:
sinasc_raw.dtypes

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResUf       object
munResLat     float64
munResLon     float64
munResAlt     float64
munResArea    float64
Length: 69, dtype: object

In [7]:
sinasc_raw.select_dtypes(exclude=['int64','float64'])

,ESCMAE,GESTACAO,GRAVIDEZ,PARTO,DTNASC,SEXO,RACACOR,DTCADASTRO,CODANOMAL,VERSAOSIST,DTRECEBIM,munResStatus,munResTipo,munResNome,munResUf
0,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-19,Masculino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
1,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-21,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
2,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-25,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
3,12 anos ou mais,37 a 41 semanas,Única,Cesáreo,2019-03-20,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia
4,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-03-23,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,12 anos ou mais,32 a 36 semanas,Única,Cesáreo,2019-12-13,Masculino,Branca,2020-09-28,NaN,3.2.01,2020-10-13,ATIVO,MUNIC,Vilhena,Rondônia
27024,8 a 11 anos,37 a 41 semanas,Única,Vaginal,2019-10-04,Masculino,Branca,2019-10-16,NaN,3.2.01,2019-10-21,ATIVO,MUNIC,Chupinguaia,Rondônia
27025,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-08-02,Masculino,Parda,2019-08-07,NaN,3.2.00,2019-08-07,ATIVO,MUNIC,Vilhena,Rondônia
27026,8 a 11 anos,32 a 36 semanas,Única,Vaginal,2019-12-23,Masculino,Parda,2020-01-07,NaN,3.2.00,2020-01-07,ATIVO,MUNIC,Vilhena,Rondônia


In [4]:
sinasc_raw.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')